In [1]:
from Outils import *
from Metriques import *
from random import *
from statistics import *
from KMode import *

## Utilisation 

### Bank Data

In [2]:
Bank = pd.read_csv("Données/bank.csv",sep=";")
X = Bank[['age','job', 'marital', 'education', 'default', 'housing', 
'loan','contact','month','poutcome']]
Y = Bank['y']

In [3]:
X.head()

,age,job,marital,education,default,housing,loan,contact,month,poutcome
0,30,unemployed,married,primary,no,no,no,cellular,oct,unknown
1,33,services,married,secondary,no,yes,yes,cellular,may,failure
2,35,management,single,tertiary,no,yes,no,cellular,apr,failure
3,30,management,married,tertiary,no,yes,yes,unknown,jun,unknown
4,59,blue-collar,married,secondary,no,yes,no,unknown,may,unknown


In [4]:
centers, cluster, label = k_modes(2,X,15)

In [5]:
len(label)

4521

In [6]:
len(Y)

4521

### Comparaisons des données

In [7]:
rd = []
acc = []
adj_rd = []
for exp in range(10):
    seed(exp)
    _,_,label = k_modes(2,X,10)
    acc.append(Accuracy(Y,label))
    rd.append(rand_index(Y,label))
    adj_rd.append(Adjusted_Rand_Index(Y,label))

In [8]:
mean(rd),pstdev(rd)

(0.615000205530533, 0.0)

In [9]:
mean(acc),pstdev(acc)

(0.7398805573988055, 0.0)

In [10]:
mean(adj_rd),pstdev(adj_rd)

(-0.012295523571139855, 0.0)

### Vote

In [30]:
Df = pd.read_csv("Données/house-votes-84.csv",sep=",")
X = Df.drop(['republican'],axis=1)
Y = Df['republican']

In [31]:
rd = []
acc = []
adj_rd = []
for exp in range(1,10):
    seed(exp)
    _,_,label = k_modes(2,X,10)
    acc.append(Accuracy(Y,label))
    rd.append(rand_index(Y,label))
    adj_rd.append(Adjusted_Rand_Index(Y,label))

In [32]:
mean(acc),pstdev(acc)

(0.8663594470046083, 0.0)

In [33]:
mean(rd),pstdev(rd)

(0.7679037047285576, 0.0)

In [34]:
mean(adj_rd),pstdev(adj_rd)

(0.5358539285810952, 0.0)

### Credit Approval

In [35]:
Df = pd.read_csv("Données/credit-approval.csv",sep=",")

In [36]:
Df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,class
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [37]:
X = Df.drop(['class'],axis=1)
Y = Df['class']

In [38]:
X.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0


In [39]:
center,cluster,label =  k_modes(2,X,100)

In [40]:
Sil_KModes(cluster)

0.08499188977920702

In [41]:
rd = []
acc = []
adj_rd = []
for exp in range(1,10):
    seed(exp)
    _,_,label = k_modes(2,X,10)
    acc.append(Accuracy(Y,label))
    rd.append(rand_index(Y,label))
    adj_rd.append(Adjusted_Rand_Index(Y,label))

In [42]:
mean(acc),pstdev(acc)

(0.49420289855072463, 0.0)

In [43]:
mean(rd),pstdev(rd)

(0.49934162091668244, 0.0)

In [44]:
mean(adj_rd),pstdev(adj_rd)

(-0.006740506882226061, 0.0)